In [1]:
import pandas as pd
import numpy as np
import csv
import json
import glob
import os
import shutil
import random
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
attributes = pd.read_csv('/Users/thaophan/Documents/Flatiron/Phase5_Project/product_attributes.csv')

In [3]:
attributes

,Unnamed: 0,id,brand,gender,year,season,color,usage,division,productgroup,producttype,variantname,productdisplayname
0,0,57896,Femella,Women,2012.0,Summer,Off White,Casual,Apparel,Bottomwear,Shorts,Summer Pleated Short,Femella Women Off White Shorts
1,1,37297,CASIO,Men,2016.0,Winter,Black,Casual,Accessories,Watches,Watches,CASIO,Casio Enticer Men Black Analogue Watches (A504...
2,2,3721,Lee,Men,2011.0,Summer,Black,Casual,Accessories,Belts,Belts,Lee mens black canvas belt,Lee Men Black Canvas Belt
3,3,51041,Enamor,Women,2017.0,Summer,White,Casual,Apparel,Innerwear,Briefs,DRESSUP02,Enamor Women Ivory Coloured Briefs
4,4,2833,Lee Cooper,Men,2012.0,Summer,Black,Casual,Footwear,Shoes,Casual Shoes,Lee cooper black Casual Leather shoes Darknight,Lee Cooper Men's Casual Leather Darknight Blac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44441,44441,35401,Stens by Enroute,Men,2012.0,Summer,Brown,Casual,Footwear,Shoes,Casual Shoes,Casual Shoe,Stens by Enroute Men Brown Shoes
44442,44442,48131,Fossil,Women,2012.0,Summer,Pink,Casual,Accessories,Bags,Handbags,CONVERTIBLE MINI BERRY,Fossil Women Pink Bags
44443,44443,7960,Puma,Men,2011.0,Fall,Green,Sports,Apparel,Topwear,Tshirts,Large Logo Graphic Tee,Puma Men's Large Logo Graphic Green T-shirt
44444,44444,54112,Rocia,Women,2012.0,Winter,Black,Casual,Footwear,Shoes,Flats,ROCIA BEIGE Sandal,Rocia Women Black Flats


In [4]:
producttype_counts = attributes.producttype.value_counts()
producttype_counts_df = producttype_counts.reset_index()
producttype_counts_df.columns = ['producttype', 'count']
producttype_counts_df

,producttype,count
0,Tshirts,7070
1,Shirts,3217
2,Casual Shoes,2846
3,Watches,2542
4,Sports Shoes,2036
...,...,...
138,Ipad,1
139,Suits,1
140,Mens Grooming Kit,1
141,Shoe Laces,1


In [5]:
producttype_counts_df[producttype_counts_df['count'] >=930] 

,producttype,count
0,Tshirts,7070
1,Shirts,3217
2,Casual Shoes,2846
3,Watches,2542
4,Sports Shoes,2036
5,Kurtas,1844
6,Tops,1762
7,Handbags,1759
8,Heels,1323
9,Sunglasses,1073


In [6]:
class_list = producttype_counts_df[producttype_counts_df['count'] >=930].drop(index=6)
class_list

,producttype,count
0,Tshirts,7070
1,Shirts,3217
2,Casual Shoes,2846
3,Watches,2542
4,Sports Shoes,2036
5,Kurtas,1844
7,Handbags,1759
8,Heels,1323
9,Sunglasses,1073
10,Wallets,936


In [7]:
class_list['Class'] = class_list['producttype'].map({'Tshirts':0,'Shirts':1,'Casual Shoes':2,'Watches':3,'Sports Shoes':4,'Kurtas':5,'Handbags':6,'Heels':7,'Sunglasses':8,'Wallets':9})

In [14]:
class_list

,producttype,count,Class
0,Tshirts,7070,0
1,Shirts,3217,1
2,Casual Shoes,2846,2
3,Watches,2542,3
4,Sports Shoes,2036,4
5,Kurtas,1844,5
7,Handbags,1759,6
8,Heels,1323,7
9,Sunglasses,1073,8
10,Wallets,936,9


In [15]:
class_list.to_csv('class_table.csv')

In [8]:
class_df = attributes[attributes['producttype'].isin(list(class_list.producttype.unique()))][['id','producttype']]
class_df

,id,producttype
1,37297,Watches
4,2833,Casual Shoes
5,29289,Shirts
6,49888,Watches
7,23684,Wallets
...,...,...
44440,19094,Kurtas
44441,35401,Casual Shoes
44442,48131,Handbags
44443,7960,Tshirts


In [9]:
class_df.loc[:,'image_id'] = class_df['id'].astype(str) + ".jpg"
class_df

,id,producttype,image_id
1,37297,Watches,37297.jpg
4,2833,Casual Shoes,2833.jpg
5,29289,Shirts,29289.jpg
6,49888,Watches,49888.jpg
7,23684,Wallets,23684.jpg
...,...,...,...
44440,19094,Kurtas,19094.jpg
44441,35401,Casual Shoes,35401.jpg
44442,48131,Handbags,48131.jpg
44443,7960,Tshirts,7960.jpg


In [10]:
class_0_tshirt_list = list(class_df[(class_df['producttype']=="Tshirts")]['image_id'])
class_1_shirt_list = list(class_df[(class_df['producttype']=="Shirts")]['image_id'])
class_2_casualshoes_list = list(class_df[(class_df['producttype']=="Casual Shoes")]['image_id'])
class_3_watches_list = list(class_df[(class_df['producttype']=="Watches")]['image_id'])
class_4_sportsshoes_list = list(class_df[(class_df['producttype']=="Sports Shoes")]['image_id'])
class_5_kurtas_list = list(class_df[(class_df['producttype']=="Kurtas")]['image_id'])
class_6_handbags_list = list(class_df[(class_df['producttype']=="Handbags")]['image_id'])
class_7_heels_list = list(class_df[(class_df['producttype']=="Heels")]['image_id'])
class_8_sunglasses_list = list(class_df[(class_df['producttype']=="Sunglasses")]['image_id'])
class_9_wallets_list = list(class_df[(class_df['producttype']=="Wallets")]['image_id'])

In [11]:
class_list_all = [class_0_tshirt_list,
                  class_1_shirt_list,
                  class_2_casualshoes_list,
                  class_3_watches_list,
                  class_4_sportsshoes_list,
                  class_5_kurtas_list,
                  class_6_handbags_list,
                  class_7_heels_list,
                  class_8_sunglasses_list,
                  class_9_wallets_list]

In [12]:
class_folder_all =['/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/0',
                  '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/1',
                  '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/2',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/3',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/4',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/5',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/6',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/7',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/8',
                   '/Users/thaophan/Documents/Flatiron/Phase5_Project/Class/9',
]

In [13]:

# # Folder paths for Train, Test, and Validation
# source_folder = "/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/Source_image"
# folder1 = "/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/folder1"
# folder2 = "/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/folder2"
# folder3 = "/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/folder3"


# # Create the folder structure
# if not os.path.exists(source_folder):
#     os.makedirs(source_folder)
# if not os.path.exists(folder1):
#     os.makedirs(folder1)
# if not os.path.exists(folder2):
#     os.makedirs(folder2)
# if not os.path.exists(folder3):
#     os.makedirs(folder3)

# # Ratio of data split
# source_ratio = 0.02
# folder1_ratio = 0.05
# folder2_ratio = 0.1
# folder2_ratio = 1 - folder_ratio - folder1_ratio - folder2_ratio

# # Loop through each class folder
# for class_folder in class_folder_all:
#     # Get the list of images in the current class folder
#     image_list = os.listdir(class_folder)

#     # Shuffle the list of images
#     random.shuffle(image_list)

#     # Calculate the split points based on the ratios
#     source_split = int(len(image_list) * source_ratio)
#     folder1_split = source_split + int(len(image_list) * folder1_ratio)
#     folder2_split = folder1_split + int(len(image_list) * folder2_ratio)

#     # Split the image list into Train, Test, and Validation
#     source_data = image_list[:source_split]
#     folder1_data = image_list[source_split:folder1_split]
#     folder2_data = image_list[folder1_split:folder2_split]
#     folder3_data = image_list[folder2_split:]

#     # Function to move images to their corresponding folder
#     def move_images(image_list, src_folder, dest_folder):
#         for image_name in image_list:
#             src_path = os.path.join(src_folder, image_name)
#             dest_path = os.path.join(dest_folder, image_name)
#             shutil.copy(src_path, dest_path)

#     # Move images to their corresponding folders
#     class_name = os.path.basename(class_folder)
#     source_class_folder = os.path.join(source_folder, class_name)
#     folder1_class_folder = os.path.join(folder1, class_name)
#     folder2_class_folder = os.path.join(folder2, class_name)
#     folder3_class_folder = os.path.join(folder3, class_name)

#     if not os.path.exists(source_class_folder):
#         os.makedirs(source_class_folder)
#     if not os.path.exists(folder1_class_folder):
#         os.makedirs(folder1_class_folder)
#     if not os.path.exists(folder2_class_folder):
#         os.makedirs(folder2_class_folder)
#     if not os.path.exists(folder3_class_folder):
#         os.makedirs(folder3_class_folder)

#     move_images(source_data, class_folder, source_class_folder)
#     move_images(folder1_data, class_folder, folder1_class_folder)
#     move_images(folder2_data, class_folder, folder2_class_folder)
#     move_images(folder3_data, class_folder, folder3_class_folder)


In [28]:
# def copy_random_images(source_folder, destination_folder, num_images=2):
#     # Get the list of class folders in the source folder
#     class_folders = [folder for folder in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, folder))]
    
#     # Loop through each class folder
#     for class_folder in class_folders:
#         class_path = os.path.join(source_folder, class_folder)
#         image_files = [file for file in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, file))]
        
#         # Select random images (num_images) from the class folder
#         random_images = random.sample(image_files, num_images)
        
#         # Copy the random images to the destination folder
#         for image in random_images:
#             src_path = os.path.join(class_path, image)
#             dst_path = os.path.join(destination_folder, image)
#             shutil.copy(src_path, dst_path)

# # Source folder path
# source_folder_path = '/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/Source_image'

# # Destination folder path to store random images
# destination_folder_path = '/Users/thaophan/Documents/Flatiron/Phase5_Project/Deployment/Uploaded_image'

# # Number of random images to select from each class folder
# num_random_images = 2

# # Call the function to copy random images
# copy_random_images(source_folder_path, destination_folder_path, num_random_images)
